# Machine Learning - Hands on Lab - Session #1

* **Lecturer:** Jonathan DEKHTIAR
* **Date:** 2017-03-13
<br/><br/>
* **Contact:** [contact@jonathandekhtiar.eu](mailto:contact@jonathandekhtiar.eu)
* **Twitter:** [@born2data](https://twitter.com/born2data)
* **LinkedIn:** [JonathanDEKHTIAR](https://fr.linkedin.com/in/jonathandekhtiar)
* **Personal Website:** [JonathanDEKHTIAR](http://www.jonathandekhtiar.eu)
* **RSS Feed:** [FeedCrunch.io](https://www.feedcrunch.io/@dataradar/)
* **Tech. Blog:** [born2data.com](http://www.born2data.com/)
* **Github:** [DEKHTIARJonathan](https://github.com/DEKHTIARJonathan)
<br/><br/>

```
*************************************************************************
**
** 2017 March 13
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

# Let's clean the data buddy !

With that general overview out of the way, let's start cleaning the Airbnb data. 

In relation to the datasets provided for the Airbnb Kaggle competition, we will focus our cleaning efforts on two files:

* **train_users_2.csv** 
* **test_users.csv** 

For now, *sessions.csv* will be left aside.

## 1. Loading the Python libraries


In [1]:
import os

import numpy as np
import pandas as pd

## 2. Loading in the Data

The first step is to load the data from the CSV files using Python. 
To do this we will use the Pandas library and load the data from two files **train_users_2.csv** and **test_users.csv**. 

After loading, we will combine them into one dataset so that any cleaning (and later any other changes) will be done to all the data at once.

In [2]:
df_train = pd.read_csv("data/train_users_2.csv")
df_train.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
97279,xzlqc2fczu,2013-08-07,20130807053513,2013-08-12,FEMALE,NaN,basic,0,en,direct,direct,untracked,Web,Windows Desktop,Chrome,US
4115,k73c8hcxz3,2011-04-09,20110409013949,2011-05-05,-unknown-,29.0,facebook,2,en,other,craigslist,untracked,Web,Windows Desktop,Chrome,CA
115501,b3blqt4vqc,2013-10-09,20131009155411,2013-11-07,-unknown-,48.0,basic,25,en,direct,direct,untracked,iOS,iPhone,-unknown-,US
86055,hfi0yii255,2013-06-24,20130624033933,NaN,FEMALE,49.0,facebook,0,en,direct,direct,untracked,Web,iPad,Mobile Safari,NDF
144105,aqsjjomf53,2014-01-20,20140120221144,NaN,MALE,24.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF


In [3]:
df_test = pd.read_csv("data/test_users.csv")
df_test.sample(n=5) # Only display a few lines and not the whole dataframe

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
31345,pdlmlrigjc,2014-08-14,20140814154442,NaN,FEMALE,27.0,facebook,0,es,sem-brand,google,omg,Web,Windows Desktop,Chrome
47210,mwkj5y3bge,2014-09-07,20140907011254,NaN,-unknown-,NaN,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
34548,dtmczaohp0,2014-08-19,20140819055848,NaN,MALE,28.0,basic,0,en,direct,direct,untracked,Moweb,Android Phone,Android Browser
5802,5xtwbkhiz3,2014-07-11,20140711031536,NaN,-unknown-,NaN,basic,0,en,direct,direct,linked,Moweb,Android Phone,Chrome Mobile
337,tbnsmr9il1,2014-07-01,20140701163704,NaN,MALE,34.0,basic,0,en,direct,direct,linked,Web,Mac Desktop,Chrome


In [4]:
# Combine into one dataset
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105


## 3. Cleaning the timestamp

Once the data has been loaded and combined, the first cleaning step we will undertake is fixing the format of the dates.

Fixing date format is essential in order to **simplify** date manipulation.

In [5]:
df_all['date_account_created'] = pd.to_datetime(df_all['date_account_created'], format='%Y-%m-%d')
df_all['timestamp_first_active'] = pd.to_datetime(df_all['timestamp_first_active'], format='%Y%m%d%H%M%S')

df_all.head(n=5) # Only display a few lines and not the whole dataframe

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,2009-03-19 04:32:55
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,2009-05-23 17:48:09
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,2009-06-09 23:12:47
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,2009-10-31 06:01:29
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,2009-12-08 06:11:05


## 4. Remove booking date field

For every test data the column "date_first_booking" is missing, so we can't rely on this one for our prediction model.

**=> We must delete the column**

In [6]:
# Remove date_first_booking column
df_all.drop('date_first_booking', axis=1, inplace=True)

## 5. Clean the Age column

There are several age values that are clearly incorrect (unreasonably high or too low). 

In this step, we replace these incorrect values with "NaN", which literally stands for *Not a Number*, but implies we do not know the age value. In other words we are changing the incorrect values into missing values. 

In [7]:
# Remove outliers function
def remove_outlier(x, min_val=15, max_val=90):
    if np.isnan(x):
        return np.nan
    elif np.logical_or(x<=min_val, x>=max_val):
        return np.nan
    else:
        return x

In [8]:
# Fixing age column
df_all['age'] = df_all['age'].apply(lambda x: remove_outlier(x, min_val=15, max_val=90))
df_all['age'].fillna(-1, inplace=True)

df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
208153,direct,direct,25.0,US,2014-06-21,untracked,Firefox,Mac Desktop,MALE,28iy0ly0bj,en,Web,0,facebook,2014-06-21 09:31:56
59291,direct,direct,24.0,NDF,2013-02-06,untracked,Chrome,Mac Desktop,MALE,291erqfv17,en,Web,0,facebook,2013-02-06 18:10:38
152654,direct,direct,41.0,FR,2014-02-13,untracked,Chrome,Mac Desktop,FEMALE,7jlh8y96b9,en,Web,0,basic,2014-02-13 22:40:42
70675,api,other,27.0,NDF,2013-04-13,untracked,-unknown-,iPhone,FEMALE,scnvq6fye4,en,iOS,12,facebook,2013-04-13 01:13:09
2543,direct,direct,-1.0,US,2010-12-02,NaN,-unknown-,Other/Unknown,FEMALE,4wixfhqtzf,en,Web,0,basic,2010-12-02 02:02:42


As mentioned earlier, there are several more complicated ways to fill in the missing values in the age column. 

We are selecting this simple method for two main reasons:

* **Clarity**: this workshop is going to be long enough without adding the complication of a complex methodology for imputing missing ages.

* **Questionable results**: in my testing during the actual competition, I did test several more complex imputation methodologies. However, none of the methods I tested actually produced a better end result than the methodology outlined above.


## 6. Converting Age Column to Integers

Values inside the age columns are **floats**, it is better to have them as **integers**

In [9]:
df_all.age = df_all.age.astype(int)
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
98714,direct,direct,-1,NDF,2013-08-12,untracked,Safari,Mac Desktop,-unknown-,pqq5j5hidw,en,Web,0,basic,2013-08-12 19:10:56
235645,sem-brand,google,45,NaN,2014-08-01,omg,IE,Windows Desktop,FEMALE,a8je0n2g3j,en,Web,0,basic,2014-08-01 19:09:13
209380,direct,direct,-1,NDF,2014-06-23,untracked,Chrome Mobile,Android Phone,-unknown-,mobm7ehrm0,en,Moweb,0,basic,2014-06-23 21:47:54
273254,direct,direct,-1,NaN,2014-09-27,untracked,Chrome Mobile,Android Phone,-unknown-,9e5qum4w5i,en,Android,23,basic,2014-09-27 05:33:48
85847,direct,direct,-1,NDF,2013-06-23,untracked,Firefox,Mac Desktop,-unknown-,scdz1guxv7,en,Web,0,basic,2013-06-23 05:41:27


## 7. Identify and fill additional columns with missing values

In [10]:
def check_NaN_Values_in_df(df):
    # searching for NaN values is all the columns
    for col in df:
        nan_count = df[col].isnull().sum()

        if nan_count != 0:
            print (col + " => "+  str(nan_count) + " NaN Values")
            
check_NaN_Values_in_df(df_all) 

country_destination => 62096 NaN Values
first_affiliate_tracked => 6085 NaN Values


It is **absolutely normal** that <i>country_destination</i> contains NaN values, it is the output field. NaN Values are test data.

However, **we need to fix** the column <i>first_affiliate_tracked</i> 

In [11]:
df_all['first_affiliate_tracked'].fillna(-1, inplace=True)

# We check there is no more NaN except in country_destination column
check_NaN_Values_in_df(df_all) 

df_all.sample(n=5)

country_destination => 62096 NaN Values


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
140602,other,padmapper,31,NDF,2014-01-11,tracked-other,Chrome,Mac Desktop,FEMALE,z4c7t8p8jn,en,Web,0,facebook,2014-01-11 19:37:22
48957,direct,direct,-1,NDF,2012-11-18,untracked,Mobile Safari,iPhone,-unknown-,mus21jee9f,en,Web,0,basic,2012-11-18 01:02:14
117129,direct,direct,37,NDF,2013-10-15,linked,Chrome,Windows Desktop,MALE,utg1gr6non,en,Web,0,basic,2013-10-15 23:06:15
142229,direct,direct,-1,NDF,2014-01-15,untracked,Firefox,Windows Desktop,-unknown-,lhtalzjjye,en,Web,0,basic,2014-01-15 20:47:28
60939,sem-non-brand,vast,-1,NDF,2013-02-18,omg,Firefox,Mac Desktop,-unknown-,p54kgtkzkf,en,Web,0,basic,2013-02-18 00:51:29


## 8. Removing Users which registred before Feb. 2013

In [12]:
df_all = df_all[df_all['date_account_created'] > '2013-02-01']
df_all.sample(n=5)

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
132982,direct,direct,-1,US,2013-12-16,untracked,Firefox,Windows Desktop,-unknown-,qc6w4nzs6m,en,Web,0,basic,2013-12-16 23:03:56
156708,direct,direct,-1,NDF,2014-02-25,-1,-unknown-,Other/Unknown,-unknown-,vjm5e8lzo6,en,iOS,25,basic,2014-02-25 08:10:05
244192,direct,direct,38,NaN,2014-08-13,untracked,IE,Windows Desktop,MALE,2813vaojmt,en,Web,0,basic,2014-08-13 19:27:22
202547,seo,facebook,-1,other,2014-06-10,untracked,IE,Windows Desktop,-unknown-,0hjij5mpba,en,Web,0,basic,2014-06-10 18:43:44
143304,direct,direct,-1,GB,2014-01-18,linked,Safari,Mac Desktop,-unknown-,dxvop5t8lb,en,Web,0,basic,2014-01-18 05:31:38


## 9. Saving the DataFrame to csv

In [13]:
# We create the output directory if necessary
if not os.path.exists("output"):
    os.makedirs("output")
    
# We export to csv
df_all.to_csv("output/cleaned.csv", sep=',', index=False)